In [11]:
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import datetime
import time
import boto3
import io

In [ ]:
# Definición de las variables globales
TMDB_KEY = "af1e6bbca0680fe32c9669b0c313dca6"
LATEST_MOVIE = f"https://api.themoviedb.org/3/movie/latest?api_key={TMDB_KEY}&language=en-US"
MOVIES_URL = f"https://api.themoviedb.org/3/movie"
GENRE_URL = f"https://api.themoviedb.org/3/genre/movie/list?api_key={TMDB_KEY}&language=en-US"
CHANGES_URL = f"https://api.themoviedb.org/3/movie/changes?api_key={TMDB_KEY}&language=en-US"

In [3]:
current_date = datetime.today().strftime("%Y-%m-%d")

In [8]:
BUCKET_NAME = "data-lake-tmdb"
session = boto3.Session()
s3 = session.client('s3')

In [ ]:
try:
    # Obtención del ID de la última película
    response = requests.get(LATEST_MOVIE)
    if response.status_code == 200:
        data = response.json()
        last_movie_id = data["id"]
        print("ID de la última película:", last_movie_id)
    else:
        print(f"Error en la solicitud: {response.status_code}")
        last_movie_id = None

    j = 1_000_000  # Punto de inicio para la descarga de películas

    while last_movie_id and j <= last_movie_id:
        movies = []
        print(f"\nDescargando películas desde el id {j} hasta {j+9999}...")

        for i in range(10_000):
            current_id = j + i
            print(f"Solicitando id: {current_id}", end="\r")
            try:
                response = requests.get(MOVIES_URL + f"/{current_id}?api_key={TMDB_KEY}&language=en-US", timeout=10)
                if response.status_code == 200:
                    data = response.json()
                    movies.append(data)
                else:
                    print(f"Error en la solicitud para id {current_id}: {response.status_code}")
            except requests.exceptions.Timeout:
                print(f"Timeout en la solicitud para id {current_id}")
            except Exception as e:
                print(f"Error inesperado en id {current_id}:", e)
            time.sleep(0.05)

        file_name = f"movies{j}.json"
        print(f"\nGuardando lote en {file_name} con {len(movies)} películas.")

        # Guardar el archivo en un buffer para subir a S3
        buffer = io.BytesIO()
        buffer.write(json.dumps(movies, ensure_ascii=False, indent=4).encode('utf-8'))
        buffer.seek(0)

        try:
            s3.put_object(Bucket=BUCKET_NAME, Key=file_name, Body=buffer)
            print(f"Archivo {file_name} subido a S3 bucket {BUCKET_NAME}")
        except Exception as e:
            print(f"Error al subir {file_name} a S3:", e)

        # Guardar el último id en buffer para subir a S3
        last_movie_buffer = io.BytesIO()
        last_movie_buffer.write(str(current_id).encode('utf-8'))
        last_movie_buffer.seek(0)
        try:
            s3.put_object(Bucket=BUCKET_NAME, Key="last-movie-saved.txt", Body=last_movie_buffer)
            print(f"Última película guardada en S3: {current_id}")
        except Exception as e:
            print("Error al guardar el último id en S3:", e)

        if j >= last_movie_id:
            print("Fin de la descarga.")
            break

        j += 10_000

except Exception as e:
    print("Error global inesperado:", e)

In [14]:
try:
    response = requests.get(GENRE_URL, timeout=10)

    if response.status_code == 200:
        data = response.json()
        print("Géneros extraídos correctamente.")

        # Guardar en buffer y subir a S3
        genre_buffer = io.BytesIO()
        genre_buffer.write(json.dumps(data, ensure_ascii=False, indent=4).encode('utf-8'))
        genre_buffer.seek(0)

        s3.put_object(Bucket=BUCKET_NAME, Key="genres.json", Body=genre_buffer)
        print("Archivo 'genres.json' subido a S3 bucket", BUCKET_NAME)
        
    else:
        print(f"Error en la solicitud: {response.status_code}")
except requests.exceptions.Timeout:
    print("Timeout en la solicitud de géneros.")
except Exception as e:
    print("Error inesperado:", e)

Géneros extraídos correctamente.
Archivo 'genres.json' subido a S3 bucket data-lake-tmdb
